In [3]:
!pip install catboost -q
!pip install lightgbm -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.7 MB/s eta 0:00:00


In [4]:
import zipfile
import os
import shutil
import random
import gdown

import cv2
from PIL import Image, ImageOps

from skimage.feature import hog

from sklearn.model_selection import train_test_split, GridSearchCV, KFold, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

from tqdm.auto import tqdm

In [5]:
RANDOM_STATE = 42
random.seed(RANDOM_STATE)

In [7]:
try:
    from google.colab import drive

    drive.mount("/content/drive")
    DRIVE_DIR = os.path.join("/content/drive", "MyDrive")
except ImportError:
    DRIVE_DIR = os.getcwd()

DATASET_DIR = os.path.join(os.getcwd(), "dataset")
TRAIN_DIR = os.path.join(DATASET_DIR, "train")
TEST_DIR = os.path.join(DATASET_DIR, "test")

TEMP_DIR = os.path.join(os.getcwd(), "temp")
TEMP_TRAIN_DIR = os.path.join(TEMP_DIR, "train")
TEMP_TEST_DIR = os.path.join(TEMP_DIR, "test")

ZIP_PATH = os.path.join(DRIVE_DIR, "dataset_32_classes_splitted.zip")
os.makedirs(DATASET_DIR, exist_ok=True)

Mounted at /content/drive


In [8]:
file_id = "1-1ehpRd0TnwB1hTHQbFHzdf55SrIri4f"
if os.path.exists(ZIP_PATH):
    print("Архив уже добавлен")
else:
    gdown.download(
        f"https://drive.google.com/uc?id={file_id}", os.path.join(os.getcwd(), "dataset_32_classes.zip"), quiet=False
    )

Архив уже добавлен


In [9]:
# Распаковка архива
with zipfile.ZipFile(ZIP_PATH, "r") as zip_ref:
    zip_ref.extractall("./dataset")

In [10]:
classes = os.listdir(TRAIN_DIR)

# Проверим структуру папок
print(f"Количество папок: {len(classes)}")

Количество папок: 32


In [11]:
assert len(classes) == len(os.listdir(TEST_DIR))

In [12]:
def resize_image(image, size: tuple[int, int]):
    img = Image.fromarray(image)
    if img.mode != "RGB":
        img = img.convert("RGB")
    ratio = img.width / img.height
    # Широкое изображение
    if ratio > 1:
        new_width = size[0]
        new_height = int(size[0] / ratio)
    # Высокое изображение
    else:
        new_height = size[1]
        new_width = int(size[1] * ratio)
    img_resized = img.resize((new_width, new_height), Image.LANCZOS)
    img_padded = ImageOps.pad(img_resized, size, color="white", centering=(0.5, 0.5))
    return np.array(img_padded)

In [13]:
def set_image_size(img_path: str, save_path: str, size: tuple[int, int]):
    img = Image.open(img_path)
    if img.mode != "RGB":
        img = img.convert("RGB")
    ratio = img.width / img.height
    # Широкое изображение
    if ratio > 1:
        new_width = size[0]
        new_height = int(size[0] / ratio)
    # Высокое изображение
    else:
        new_height = size[1]
        new_width = int(size[1] * ratio)
    img_resized = img.resize((new_width, new_height), Image.LANCZOS)
    img_padded = ImageOps.pad(img_resized, size, color="white", centering=(0.5, 0.5))
    img_padded.save(save_path)

In [14]:
def create_resized_dataset(size: tuple[int, int], random_indexes: list[int] | None, dataset_path: str, temp_path: str):
    # Если папка уже была, то удалить из нее прошлое содержимое
    if os.path.exists(temp_path):
        shutil.rmtree(temp_path)
    os.mkdir(temp_path)

    for cl in tqdm(classes):

        temp_cl_path = os.path.join(temp_path, cl)
        if os.path.exists(temp_cl_path) == False:
            os.mkdir(temp_cl_path)

        folder_path = os.path.join(dataset_path, cl)
        image_names = os.listdir(folder_path)
        if random_indexes is not None:
            image_names = [image_names[i] for i in random_indexes]
        for img_name in image_names:
            img_path = os.path.join(dataset_path, cl, img_name)
            save_path = os.path.join(temp_path, cl, img_name)
            set_image_size(img_path, save_path, size)

In [17]:
# Для тестовой выборки возьму 500 изображений из 1120. random всегда выдает разные значения, поэтому индексы запомню одни для всех экспериментов
random_indexes = random.sample([i for i in range(0, 1120)], 500)

In [19]:
if not os.path.exists(TEMP_DIR):
    os.mkdir(TEMP_DIR)
create_resized_dataset((64, 64), random_indexes, TRAIN_DIR, TEMP_TRAIN_DIR)
create_resized_dataset((64, 64), range(0, 280), TEST_DIR, TEMP_TEST_DIR)

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

In [23]:
def load_colored_images_and_labels(dataset_path: str):
    images = []
    labels = []
    classes = os.listdir(dataset_path)
    for class_label in tqdm(classes):
        class_folder = os.path.join(dataset_path, class_label)
        for file in os.listdir(class_folder):
            file_path = os.path.join(class_folder, file)
            img = cv2.imread(file_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Преобразуем в RGB
            images.append(img)
            labels.append(class_label)
    return np.array(images), np.array(labels)

In [24]:
X_train, y_train = load_colored_images_and_labels(TEMP_TRAIN_DIR)
X_test, y_test = load_colored_images_and_labels(TEMP_TEST_DIR)

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

Для HOG

In [38]:
def extract_hog_color_features(images, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), size=(64, 64)):
    hog_features = []
    for image in images:
        img_hog_features = []
        resized_image = resize_image(image, size)
        for channel in cv2.split(resized_image):
            features = hog(
                channel,
                orientations=orientations,
                pixels_per_cell=pixels_per_cell,
                cells_per_block=cells_per_block,
                block_norm="L2-Hys",
                visualize=False,
            )
            img_hog_features.append(features)
        hog_features.append(np.hstack(img_hog_features))
    return np.array(hog_features)

In [39]:
class HogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, orientations=3, pixels_per_cell=(10, 10), cells_per_block=(2, 2), size=(64, 64)):
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
        self.size = size

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return extract_hog_color_features(
            X,
            orientations=self.orientations,
            pixels_per_cell=self.pixels_per_cell,
            cells_per_block=self.cells_per_block,
            size=self.size,
        )

    def predict(self, X):
        return self.transform(X)

In [40]:
hog_transformer = HogTransformer(orientations=3, pixels_per_cell=(10, 10), cells_per_block=(2, 2), size=(64, 64))
pca = PCA(n_components=0.6)
catboost = CatBoostClassifier(
    depth=10, learning_rate=0.1, min_child_samples=44, reg_lambda=0.051712194163615596, random_state=42, verbose=False
)
hog_catboost = make_pipeline(hog_transformer, pca, catboost)
hog_catboost

Pipeline(steps=[('hogtransformer', HogTransformer()),
                ('pca', PCA(n_components=0.6)),
                ('catboostclassifier',
                 <catboost.core.CatBoostClassifier object at 0x7c582d5435d0>)])

Для SIFT

In [41]:
def get_SIFT_descriptors(img):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(img, None)
    return descriptors

In [42]:
def create_feature_vector(descriptors, num_features=128):
    feature_vector = np.zeros(num_features)

    if descriptors is not None and len(descriptors) > 0:
        if descriptors.shape[0] < num_features:
            feature_vector = np.mean(descriptors, axis=0)
        else:
            feature_vector = np.mean(descriptors[:num_features], axis=0)

    return feature_vector

In [43]:
def extract_sift_features(images, size=(64, 64)):
    features = []
    for img in images:
        img = resize_image(img, size)
        descriptors = get_SIFT_descriptors(img)
        feature_vector = create_feature_vector(descriptors)
        features.append(feature_vector)
    return np.array(features)

In [44]:
class SiftTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, size=(64, 64)):
        self.size = size

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return extract_sift_features(X, size=self.size)

    def predict(self, X):
        return self.transform(X)

In [45]:
sift_transformer = SiftTransformer(size=(64, 64))
pca = PCA(n_components=0.6)
lgbm = LGBMClassifier(min_child_samples=66, num_leaves=165, reg_alpha=0.00093, reg_lambda=0.00074, random_state=42)
sift_lgbm = make_pipeline(sift_transformer, pca, lgbm)
sift_lgbm

Pipeline(steps=[('sifttransformer', SiftTransformer()),
                ('pca', PCA(n_components=0.6)),
                ('lgbmclassifier',
                 LGBMClassifier(min_child_samples=66, num_leaves=165,
                                random_state=42, reg_alpha=0.00093,
                                reg_lambda=0.00074))])

Объединенная модель

In [ ]:
voting_boosting = VotingClassifier(estimators=[("hog_catboost", hog_catboost), ("sift_lgbm", sift_lgbm)], voting="soft")

voting_boosting.fit(X_train, y_train)

In [50]:
voting_boosting

VotingClassifier(estimators=[('hog_catboost',
                              Pipeline(steps=[('hogtransformer',
                                               HogTransformer()),
                                              ('pca', PCA(n_components=0.6)),
                                              ('catboostclassifier',
                                               <catboost.core.CatBoostClassifier object at 0x7c582d5435d0>)])),
                             ('sift_lgbm',
                              Pipeline(steps=[('sifttransformer',
                                               SiftTransformer()),
                                              ('pca', PCA(n_components=0.6)),
                                              ('lgbmclassifier',
                                               LGBMClassifier(min_child_samples=66,
                                                              num_leaves=165,
                                                              random_state=42,
                                                              reg_alpha=0.00093,
                                                              reg_lambda=0.00074))]))],
                 voting='soft')

In [ ]:
pred_test = voting_boosting.predict(X_test)

In [48]:
print(classification_report(y_test, pred_test))

              precision    recall  f1-score   support

       Apple       0.88      0.77      0.82       280
     Avocado       0.85      0.79      0.82       280
      Banana       0.85      0.84      0.84       280
        Bean       0.65      0.58      0.61       280
Bitter_Gourd       0.59      0.80      0.68       280
Bottle_Gourd       0.76      0.86      0.81       280
     Brinjal       0.55      0.55      0.55       280
    Broccoli       0.60      0.62      0.61       280
     Cabbage       0.61      0.54      0.57       280
    Capsicum       0.66      0.67      0.67       280
      Carrot       0.73      0.80      0.76       280
 Cauliflower       0.62      0.60      0.61       280
      Cherry       0.91      0.91      0.91       280
    Cucumber       0.86      0.74      0.80       280
       Grape       0.96      0.95      0.96       280
        Kiwi       0.78      0.70      0.74       280
       Mango       0.90      0.82      0.86       280
         Nut       0.93    

In [ ]:
pred_train = voting_boosting.predict(X_train)

In [51]:
accuracy_score(y_train, pred_train)

1.0

**Вывод:** Объединение лучших моделей для HOG и SIFT с помощью VotingClassifier немного ухудшило метрики - 0.75 для f1-macro и accuracy. Также модель оказалась переобученной и подстроилась под тренировочные данные.